In [ ]:
pip install -q yahoo_fin

In [4]:
mkdir -p data results log

In [ ]:
pip install  git+https://github.com/ahsank/runml@v0.0.5#egg=runml

In [5]:
import runml.pipeline
from runml import pipeline,findata
from google.colab import drive
#from importlib import reload
# reload(runml.pipeline)

Run following if like to save results in drive

In [6]:
drive.mount('/content/drive')
!cp  /content/drive/MyDrive/colab/results/* ./results/

Mounted at /content/drive


In [7]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accuracy']/stop_loss - (1-df['Accuracy'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
pending = ['PAR']
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRCT', 'CRWD',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DXCM',
           'ESTC', 'ETSY', 'EXPE', 'FOUR', 'GFS', 'GTLB', 'GLBE', 'GOGO',
           'INDI', 'INMD', 'INTA', 'IOT', 'IRDM', 'JKS', 'LUMN', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL',
           'MTCH',  'NVCR','OKTA', 'OLED', 'OSTK',
           'PERI', 'PANW', 'PAYO', 'PD', 'PLUG', 'PI', 'PINS', 'PTON', 'PUBM',
           'RBLX', 'SNAP', 'SNOW', 'SPLK', 'SPOT', 'SQ', 'STEM',
           'TEAM', 'TDOC', 'TOST', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df  = pipeline.runModelCombined(tickers, 'ipos-10a', mod, False, loss=lossfn)[0]

In [ ]:
addAlloc(df, 0.10, 1)
df.sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
75,ZS,RROnly,1785.19,1809.09,3594.29,43.925529,0.973404,175.04,204.600006,0.17,9.577597
30,IOT,RROnly,123.47,90.07,213.54,9.195471,0.964286,24.00,20.370001,-0.15,9.404762
5,BILL,RROnly,1920.26,1883.04,3803.30,33.844213,0.945946,56.48,71.239998,0.26,9.251559
26,GOGO,RROnly,139.76,132.72,272.49,1.742038,0.968085,10.08,9.400000,-0.07,9.224924
61,STEM,RROnly,200.86,200.65,401.51,3.542794,0.930070,2.83,4.870000,0.72,9.203574
65,TTD,RROnly,707.81,612.24,1320.06,16.476181,0.936170,64.01,89.800003,0.40,9.202128
4,APP,RROnly,344.44,470.46,814.90,10.807667,0.949153,43.79,49.570000,0.13,9.100391
64,TOST,RROnly,118.25,114.70,232.95,13.370140,0.936842,14.11,17.370001,0.23,9.093822
23,GFS,RROnly,209.80,305.42,515.22,39.850036,0.966667,51.97,54.799999,0.05,9.000000
25,GLBE,RROnly,176.14,324.81,500.95,18.337717,0.938596,35.18,39.590000,0.13,8.913630


In [ ]:
findata.EPOCHS=200
later = [ ]
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df1, res = pipeline.runModelCombined(tickers1, 'etf-11a', mod, True)


In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

(0.9202127659574468,
 0.9229203567322334,
 0.09075827015321843,
 0.004420030125699004,
 0.01842253193096099)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
12,IXN,RROnly,258.00,189.98,447.98,26.574037,0.978723,63.34,58.950001,-0.07,9.483283
3,DAPP,RROnly,93.44,111.79,205.23,2.865296,0.949153,5.87,7.180000,0.22,9.260401
37,XLK,RROnly,695.26,558.92,1254.18,69.440428,0.962766,179.52,161.490005,-0.10,9.255319
23,SMH,RROnly,809.82,485.36,1295.17,50.936014,0.968085,157.28,148.729996,-0.05,9.042553
19,MGV,RROnly,283.88,272.38,556.26,52.337136,0.952128,101.31,93.099998,-0.08,8.922872
44,XNTK,RROnly,692.46,566.37,1258.83,64.106250,0.946809,146.65,133.020004,-0.09,8.877069
2,ARKW,RROnly,687.29,775.76,1463.05,38.031804,0.930851,58.90,67.190002,0.14,8.814590
31,XITK,RROnly,932.81,934.74,1867.55,85.943111,0.952128,123.26,130.350006,0.06,8.723404
1,ARKK,RROnly,603.56,507.34,1110.90,30.811585,0.930851,39.73,43.650002,0.10,8.617021
17,JETS,RROnly,108.39,163.77,272.15,12.255693,0.920213,15.69,17.700001,0.13,8.588380


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
# exclude = ['GTSWF', 'CBNT', 'CCL', 'CREX'. 'DTEA'. 'ILAL',
# 'TNXP', 'MRIN', 'MULN', 'AHT',
# 'LILM', 'JMIA', 'RKLB', 'VRAR', 'WISH',]
tickers3 = [ 'AIRI', 'AMSC', 'APRN', 'ACHR', 'AUGX', 'AUR',   'ATOM',
            'BKKT', 'BGSF', 'BLNK', 'BSGM',
            'CARS', 'CLOV', 'CZOO',
            'DLO',  'DM', 'DMTK', 'DPSI', 'EVGO', 'EXPR', 'FSLY', 'FSR', 'FTCH',
            'GOVX', 'GRAB', 'GTE', 'HIMS', 'HIVE',
            'INLX',  'INVZ', 'JOBY',  'KIND', 'KULR',
            'LAZR', 'LUMN', 'NNOX', 'MTTR',
            'NEPH', 'NSTG', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PTON', 'RAD', 'REAL', 'REI', 'RIG', 'RKLB', 'RVYL',
            'SFIX', 'SOUN', 'STEM', 'SFT',  'TELL', 'VTEX', 'YEXT']
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df3 = pipeline.runModelCombined(tickers3, 'vols-10a', mod, False)[0]

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3.sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
41,OPEN,RROnly,122.33,155.13,277.46,1.408282,0.906250,2.45,5.040000,1.06,5.242439
38,NSTG,RROnly,444.61,537.67,982.28,2.380378,0.925532,1.28,1.690000,0.32,5.211593
40,ONDS,RROnly,30.15,74.53,104.68,0.555360,0.897059,0.91,1.530000,0.68,5.125433
45,RAD,RROnly,121.83,181.27,303.10,0.602973,0.861702,0.25,1.020000,3.02,5.023042
25,GTE,RROnly,119.99,95.80,215.79,2.129674,0.909574,6.40,5.120000,-0.20,4.898310
48,RIG,RROnly,51.75,43.91,95.66,0.849550,0.888298,6.50,4.390000,-0.32,4.876212
22,FTCH,RROnly,299.34,307.41,606.75,2.360672,0.904255,1.66,2.010000,0.21,4.863222
10,BSGM,RROnly,43.78,47.28,91.07,0.496209,0.861702,0.35,0.540000,0.53,4.807897
1,AMSC,RROnly,125.43,133.13,258.56,3.790225,0.867021,9.26,13.210000,0.43,4.790872
32,KULR,RROnly,22.15,24.13,46.28,0.358051,0.877660,0.31,0.410000,0.32,4.780390


In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX', 'TMO', 'TSLA', 'TXN', 'UNH', 'UPS']
mod4 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df4 = pipeline.runModelCombined(tickers4, 'mcap-11b', mod4, True)[0]

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
34,TSLA,RROnly,2833.04,2275.51,5108.55,29.128874,0.941489,234.30,323.730011,0.38,6.570949
4,AMZN,RROnly,978.34,804.37,1782.71,84.104791,0.957447,145.18,137.750000,-0.05,5.987842
31,QCOM,RROnly,760.26,785.55,1545.80,58.759048,0.925532,129.47,144.330002,0.11,5.933960
6,BA,RROnly,1126.93,2054.47,3181.40,99.960993,0.914894,208.04,237.220001,0.14,5.927052
27,NVDA,RROnly,2734.34,1307.11,4041.45,55.247211,0.920213,492.98,432.920013,-0.12,5.908055
33,TMO,RROnly,2423.63,1986.38,4410.01,260.327182,0.946809,466.42,493.269989,0.06,5.876393
26,NKE,RROnly,732.72,558.31,1291.03,62.515453,0.914894,105.96,119.199997,0.12,5.825735
5,AVGO,RROnly,3653.04,1866.15,5519.18,157.756436,0.909574,977.73,854.289978,-0.13,5.801379
10,CDNS,RROnly,1197.73,556.84,1754.57,57.132266,0.925532,268.27,244.110001,-0.09,5.783519
20,INTC,RROnly,227.65,306.63,534.29,24.959558,0.920213,43.81,39.980000,-0.09,5.686424


In [ ]:
findata.EPOCHS=200
tickers5 = ['AEXAF', 'ALSMY', 'AMSSY', 'ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY', 'CAAP', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX',
            'DBOEY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY', 'EQNR', 'ERIC', 'ERJ','ESEA',
            'FANUY', 'FQVLF', 'GLCNF', 'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'OZON', 'RACE', 'LYB', 'LVMUY',
            'MYTE', 'NHYDY', 'NOK', 'NU', 'NXPI', 'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS', 'THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'VWAPY', 'YARIY', 'ZURVY']
mod5 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df5 = pipeline.runModelCombined(tickers5, 'gstock-10b', mod5, False)[0]


In [ ]:
addAlloc(df5, 0.14)
df5.sort_values('Alloc', ascending=False).head(30)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
6,BAK,RROnly,115.48,136.03,251.52,3.716204,0.920213,8.13,10.050000,0.24,5.908055
25,FQVLF,RROnly,153.91,174.87,328.79,4.335041,0.877660,10.04,16.510000,0.64,5.886683
48,POAHF,RROnly,455.87,530.55,986.42,34.216975,0.909574,49.08,60.139999,0.23,5.710652
40,MYTE,RROnly,43.50,122.74,166.24,2.491741,0.916031,2.97,3.490000,0.17,5.555199
56,SE,RROnly,1335.87,1386.23,2722.10,41.307849,0.829787,37.50,59.939999,0.60,5.359676
1,ALSMY,RROnly,23.19,32.49,55.68,1.267006,0.888298,1.31,1.590000,0.22,5.329511
68,VWAPY,RROnly,95.78,78.05,173.83,6.887966,0.936170,11.60,12.700000,0.09,5.268490
53,SFTBY,RROnly,179.70,232.94,412.64,13.319336,0.904255,21.00,23.840000,0.14,5.091185
35,MGA,RROnly,402.05,413.18,815.23,22.945620,0.914894,54.35,60.529999,0.11,4.987566
24,FANUY,RROnly,97.93,100.69,198.62,11.682122,0.925532,13.64,14.850000,0.09,4.956096


In [ ]:
findata.EPOCHS=200
tickers6 = ['ARGT', 'EWD', 'FLN', 'FM',
            'ECH', 'EWA', 'EWI', 'EWC', 'EWM', 'EWP', 'EWW', 'EWY', 'EWZ',
            'IDX', 'ILF', 'KEMQ', 'THD', 'TUR', 'XCEM']
mod6 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df6 = pipeline.runModelCombined(tickers6, 'cetf-11b', mod6, True)[0]


In [ ]:
addAlloc(df6, 0.10, 1)
df6.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
11,EWW,RROnly,210.45,151.79,362.24,23.910323,0.904255,62.27,56.990002,-0.08,7.845745
10,EWP,RROnly,97.40,104.34,201.74,16.652458,0.893617,29.89,27.920000,-0.07,7.416413
7,EWI,RROnly,103.60,134.63,238.23,16.455592,0.925532,32.39,31.129999,-0.04,7.393617
15,ILF,RROnly,101.40,105.92,207.33,12.690202,0.909574,27.75,26.500000,-0.05,7.287234
9,EWM,RROnly,71.40,69.09,140.50,17.014129,0.946809,21.23,21.660000,0.02,6.808511
14,IDX,RROnly,72.43,57.53,129.95,10.100529,0.898936,16.68,17.400000,0.04,6.462766
4,FM,RROnly,79.02,104.29,183.31,18.131894,0.909574,26.88,27.580000,0.03,6.081560
8,EWC,RROnly,128.12,91.20,219.32,16.847149,0.909574,34.42,33.450001,-0.03,6.081560
13,EWZ,RROnly,192.76,161.86,354.62,16.767690,0.898936,33.76,34.770000,0.03,5.620567
16,KEMQ,RROnly,114.94,123.76,238.70,11.959819,0.840426,14.82,15.510000,0.05,5.212766


In [ ]:
findata.EPOCHS=200
review = ['AMR', 'IFF', 'BLK']
tickers7 = ['AAL', 'ALB', 'ANF', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BGFV', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'EMN',
            'F', 'FCX', 'FIS', 'FLO', 'FMS', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN', 'KD', 'KHC', 'KMB','KVUE'
             'M', 'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL', 'PXD', 'RCL', 'RH',
            'SAFE', 'SCCO', 'SPTN', 'SPG', 'STRL', 'SWK',
            'T', 'TCN', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']

mod7 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df7 = pipeline.runModelCombined(tickers7, 'val-10b', mod7, False)[0]

In [9]:
addAlloc(df7, 0.10, 1)
df7.sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
10,BX,RROnly,797.32,636.46,1433.78,32.848686,0.973404,114.28,106.360001,-0.07,9.354103
48,M,RROnly,169.90,123.62,293.53,4.511270,0.930851,17.13,13.370000,-0.22,8.994197
85,WHR,RROnly,1052.98,810.69,1863.67,58.243580,0.946809,113.68,124.790001,0.10,8.936170
67,SAFE,RROnly,458.80,694.82,1153.62,16.493625,0.898936,21.77,31.709999,0.46,8.769658
35,FMS,RROnly,130.32,225.39,355.71,12.893243,0.920213,20.72,23.459999,0.13,8.588380
68,SCCO,RROnly,382.25,242.97,625.22,20.558052,0.920213,76.74,67.949997,-0.11,8.476789
51,MPC,RROnly,704.65,351.00,1055.66,16.242032,0.920213,151.42,135.110001,-0.11,8.476789
74,TCN,RROnly,23.72,44.32,68.04,6.704079,0.916667,8.17,9.120000,0.12,8.472222
75,TEX,RROnly,408.41,250.93,659.33,12.785929,0.930851,51.52,47.279999,-0.08,8.444149
3,APO,RROnly,466.77,275.83,742.60,22.334795,0.930851,92.70,85.199997,-0.08,8.444149


In [ ]:
findata.EPOCHS=200
tickers8 = [ 'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'UUP', 'VTIP', 'WEAT']
mod8 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df8 = pipeline.runModelCombined(tickers8, 'com-12a', mod8, True)[0]

In [11]:
addAlloc(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
0,GLD,RROnly,490.01,439.94,929.95,139.709952,0.898936,192.01,182.940002,-0.05,6.968085
6,TLT,RROnly,215.39,356.95,572.34,84.065445,0.882979,92.99,96.970001,0.04,5.904255
3,SLV,RROnly,151.00,93.17,244.17,11.584324,0.893617,23.33,22.580000,-0.03,5.390071
1,ISHG,RROnly,76.43,110.75,187.18,63.729119,0.845745,70.33,70.800003,0.01,0.000000
2,PDBC,RROnly,53.12,27.19,80.31,6.687831,0.824468,14.25,14.280000,0.00,0.000000
4,SOYB,RROnly,78.02,57.72,135.74,13.651073,0.851064,27.98,28.049999,0.00,0.000000
5,TIP,RROnly,86.49,133.63,220.13,94.964277,0.845745,105.47,105.820000,0.00,0.000000
7,UUP,RROnly,37.96,30.27,68.23,24.032773,0.851064,29.20,29.209999,0.00,0.000000
8,VTIP,RROnly,29.30,14.93,44.24,40.818441,0.803191,47.75,47.340000,-0.01,0.000000
9,WEAT,RROnly,38.61,30.79,69.40,5.022168,0.824468,5.76,5.680000,-0.01,0.000000


In [ ]:
findata.EPOCHS=200
tickers9 = [ 'ADA-USD', 'BTC-USD', 'DOT-USD', 'DOGE-USD',
            'ETH-USD', 'LTC-USD', 'MATIC-USD', 'XRP-USD']
mod9 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df9 = pipeline.runModelCombined(tickers9, 'cry-11a', mod9, True)[0]

In [ ]:
addAlloc(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
6,MATIC-USD,RROnly,12.81,15.67,28.47,0.156840,0.840426,0.67,0.600000,-0.11,4.552363
1,BTC-USD,RROnly,292064.74,301948.61,594013.35,17560.039843,0.787234,34620.22,31739.240234,-0.08,2.963526
7,XRP-USD,RROnly,5.39,10.34,15.73,0.354935,0.787234,0.61,0.560000,-0.08,2.963526
4,ETH-USD,RROnly,21894.44,23240.01,45134.45,1109.326683,0.845745,1828.99,1730.510010,-0.05,2.955927
5,LTC-USD,RROnly,692.11,1890.42,2582.53,52.311932,0.781915,69.42,64.349998,-0.07,2.469605
2,DOT-USD,RROnly,183.54,322.38,505.91,4.779250,0.712766,4.61,4.880000,0.06,0.303951
0,ADA-USD,RROnly,8.49,11.95,20.44,0.318458,0.728723,0.33,0.320000,-0.01,0.000000
3,DOGE-USD,RROnly,1.88,2.15,4.03,0.061877,0.760638,0.07,0.070000,0.01,0.000000


In [12]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print

sending incremental file list
com-12a-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 3,221,232 bytes  received 35 bytes  6,442,534.00 bytes/sec
total size is 183,108,624  speedup is 56.84
/content/drive/MyDrive/colab/results/val-8b-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/val-8b-test-high-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/val-8b-test-low-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/val-8b-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/cry-9a-test-adjcl